In [5]:
import netCDF4 as nc
import rasterio
from rasterio.transform import from_origin
from rasterio.crs import CRS
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject


In [11]:
# Define input and output file paths
input_nc_file = 'data/model_output_soil_moisture.nc'
ref_raster = 'data/dead/deadwood.tif'
output_tif_file = 'data/sm_j.tif'

In [12]:
# Open the NetCDF file
nc_file = rasterio.open(input_nc_file)

# Extract metadata
meta = nc_file.meta.copy()
meta.update({
    'driver': 'GTiff',
    'height': nc_file.height,
    'width': nc_file.width,
    'crs': CRS.from_epsg(4326),  # Set coordinate system to EPSG:4326
    'transform': from_origin(nc_file.bounds.left, nc_file.bounds.top, nc_file.res[0], nc_file.res[1])
})

# Read data
data = nc_file.read(1)

# Write data to GeoTIFF
with rasterio.open(output_tif_file, 'w', **meta) as dst:
    dst.write(data, 1)

# Close the NetCDF file
nc_file.close()


# align raster

In [13]:
# Paths
input_tif_file = 'data/sm_j.tif'
ref_raster_path = 'data/dead/deadwood.tif'
aligned_output_file = 'data/sm_j_aligned.tif'

# Open the source and reference rasters
with rasterio.open(input_tif_file) as src, rasterio.open(ref_raster_path) as ref:
    # Calculate the transform and dimensions to align to the reference raster
    transform, width, height = calculate_default_transform(
        src.crs, ref.crs, ref.width, ref.height, *ref.bounds)

    # Metadata for the new file
    meta = src.meta.copy()
    meta.update({
        'crs': ref.crs,
        'transform': transform,
        'width': width,
        'height': height,
        'driver': 'GTiff'
    })

    # Reproject and write the aligned raster
    with rasterio.open(aligned_output_file, 'w', **meta) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=ref.crs,
                resampling=Resampling.nearest)


# clip variable onto deadtree shape

In [14]:
import rasterio
from rasterio.mask import mask
import numpy as np
from rasterio.features import shapes
from shapely.geometry import shape, mapping
from shapely.ops import unary_union

# Paths
ref_raster_path = 'data/dead/deadwood.tif'
aligned_output_file = 'data/sm_j_aligned.tif'
clipped_output_file = 'data/sm_j_clipped.tif'

# Create a mask polygon from the reference raster
def create_mask_polygon(raster_path):
    with rasterio.open(raster_path) as src:
        # Read the first band and use it to find areas with data
        image = src.read(1)  # Assuming data of interest is in the first band
        mask = image > 0  # Change this condition based on how your data represents "no data"
        transform = src.transform
        
        # Generate shapes from the mask
        shape_gen = shapes(image, mask=mask, transform=transform)
        polygons = [shape(geom) for geom, value in shape_gen if value]

        # Merge polygons into a single geometry if there are multiple components
        if polygons:
            merged_polygon = unary_union(polygons)
            return merged_polygon

# Use the generated polygon to clip the aligned raster
def clip_raster_with_polygon(aligned_path, output_path, polygon):
    with rasterio.open(aligned_path) as src:
        out_image, out_transform = mask(src, [mapping(polygon)], crop=True)
        out_meta = src.meta.copy()

        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        with rasterio.open(output_path, 'w', **out_meta) as out_raster:
            out_raster.write(out_image)

# Execute the functions
polygon = create_mask_polygon(ref_raster_path)
if polygon:
    clip_raster_with_polygon(aligned_output_file, clipped_output_file, polygon)
